In [1]:
#Import libraries
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
from urllib.request import urlopen, Request
import pandas as pd
import time
from bs4 import BeautifulSoup
import requests
import lxml
import json

In [2]:
headers = {
        "User-Agent":
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3538.102 Safari/537.36 Edge/18.19582"
    }

In [3]:
link = 'http://www.comidadibuteco.com.br/category/butecos/rio-de-janeiro/'
lista = [link]
pag = 2
while pag <= 9:
    link2 = link+'page/'+str(pag)+'/'
    lista.append(link2)
    pag +=1

In [4]:
info = []
data = pd.DataFrame()
j = 1
i = 0
for links in lista:
    response = requests.get(links, headers=headers).text
    soupa = BeautifulSoup(response, 'html.parser')
    results = soupa.find('div', class_= 'row text-left')
    results = results.find_all('a')
    del results[1::2]
    del results[-1]
    print('-------------------------------------------------------------------------------------------')
    print('-------------------------------PAGINA '+str(j)+'-------------------------------')
    print(links)
    print('-------------------------------------------------------------------------------------------')
    j += 1
    i=0
    for link in results:
        new = link.get('href')
        i+=1
        print('PAGINA '+str(i))
        print(new)
        
        response = requests.get(new, headers=headers).text
        soup = BeautifulSoup(response, 'html.parser')

        for container in soup.findAll('section', class_='one mecanica'):
            
            #print(container.find_all('strong')[3].text)
            #print(container.find_all('p')[2].text)
            
            #Titulo
            if container.find('h2', class_='title') != None:
                title = container.find('h2', class_='title').text
            #Nome do prato
            if container.find_all('strong')[0] != None:
                prato = container.find_all('strong')[0].text
            
            if len(container.find_all('p')[0].text.split(container.find_all('strong')[1].text+' ',1)) > 1:
                #Endereço
                if container.find_all('p')[0].text.split(container.find_all('strong')[1].text+' ',1)[1] != None:
                    endereco = container.find_all('p')[0].text.split(container.find_all('strong')[1].text+' ',1)[1]
                #Descrição
                if container.find_all('p')[0].text.replace(prato, '').replace(endereco, '').replace(container.find_all('strong')[1].text, '') != None:
                    desc = container.find_all('p')[0].text.replace(prato, '').replace(endereco, '').replace(container.find_all('strong')[1].text, '')
                #Telefone
                if container.find_all('p')[1].text.split(container.find_all('strong')[2].text+' ',1)[1] != None:
                    telefone = container.find_all('p')[1].text.split(container.find_all('strong')[2].text+' ',1)[1]
                #Funcionamento
                if container.find_all('p')[2].text.split(container.find_all('strong')[3].text+' ',1)[0] != None:
                    funcionamento = container.find_all('p')[2].text.split(container.find_all('strong')[3].text+' ',1)[0]



                
            info.append({'RESTAURANTE' : title,
                         'PRATO' : prato,
                         'DESCRICAO' : desc,
                         'ENDEREÇO' : endereco,
                         'TELEFONE' : telefone,
                         'HORARIOS' : funcionamento
                        })
            
            data = data.append(info)
            
            title = 'VAZIO'
            prato = 'VAZIO'
            desc = 'VAZIO'
            endereco = 'VAZIO'
            telefone = 'VAZIO'
            funcionamento = 'VAZIO'

-------------------------------------------------------------------------------------------
-------------------------------PAGINA 1-------------------------------
http://www.comidadibuteco.com.br/category/butecos/rio-de-janeiro/
-------------------------------------------------------------------------------------------
PAGINA 1
http://www.comidadibuteco.com.br/400-steak-club/
PAGINA 2
http://www.comidadibuteco.com.br/adega-perola/
PAGINA 3
http://www.comidadibuteco.com.br/angurme/
PAGINA 4
http://www.comidadibuteco.com.br/areias-de-itacoatiara/
PAGINA 5
http://www.comidadibuteco.com.br/armazem-sao-thiago-bar-do-gomes/
PAGINA 6
http://www.comidadibuteco.com.br/art-chopp/
PAGINA 7
http://www.comidadibuteco.com.br/baixo-araguaia/
PAGINA 8
http://www.comidadibuteco.com.br/baixo-gago/
PAGINA 9
http://www.comidadibuteco.com.br/balaio-do-ze/
PAGINA 10
http://www.comidadibuteco.com.br/balbino-delicia-de-costela/
PAGINA 11
http://www.comidadibuteco.com.br/bar-da-gema/
PAGINA 12
http://www.comid

-------------------------------------------------------------------------------------------
-------------------------------PAGINA 9-------------------------------
http://www.comidadibuteco.com.br/category/butecos/rio-de-janeiro/page/9/
-------------------------------------------------------------------------------------------
PAGINA 1
http://www.comidadibuteco.com.br/to-careca-de-saber/
PAGINA 2
http://www.comidadibuteco.com.br/umas-e-ostras/
PAGINA 3
http://www.comidadibuteco.com.br/vint-2-bar/
PAGINA 4
http://www.comidadibuteco.com.br/zeca-fc/
PAGINA 5
http://www.comidadibuteco.com.br/zecas-paqueta/
PAGINA 6
http://www.comidadibuteco.com.br/zezimbar/
PAGINA 7
http://www.comidadibuteco.com.br/zinho-bier/


In [5]:
data = data.drop_duplicates()

In [6]:
data[data['DESCRICAO']=='VAZIO']

,RESTAURANTE,PRATO,DESCRICAO,ENDEREÇO,TELEFONE,HORARIOS
3,Areias de Itacoatiara,Mix Mineiro,VAZIO,VAZIO,VAZIO,VAZIO
6,Baixo Araguaia,Entrevero Carioca,VAZIO,VAZIO,VAZIO,VAZIO
10,Bar da Gema,Ancestralidade,VAZIO,VAZIO,VAZIO,VAZIO


In [7]:
data.loc[data['RESTAURANTE']=='Areias de Itacoatiara', 'DESCRICAO'] = '2 Bolinhos de carne seca com requeijão e queijo minas, e 2 de linguiça mineira com requeijão e queijo minas. Acompanhados de chips de aipim e pesto de pimenta biquinho.'
data.loc[data['RESTAURANTE']=='Areias de Itacoatiara', 'ENDEREÇO'] = 'Rua Barão de Mesquita, 615 – Lojas C e D | Tijuca, Rio de Janeiro – RJ'
data.loc[data['RESTAURANTE']=='Areias de Itacoatiara', 'TELEFONE'] = '(21) 3549-0857'
data.loc[data['RESTAURANTE']=='Areias de Itacoatiara', 'HORARIOS'] = 'Terças, das 17h às 23hQuartas, das 17h às 23hQuintas, das 17h às 23hSextas, das 17h às 23hSábados, das 12h às 23hDomingos, das 12h às 17h'

data.loc[data['RESTAURANTE']=='Baixo Araguaia', 'DESCRICAO'] = 'Espeto com deliciosa mistura de carnes e defumados acompanhados de abacaxi, farofa de bacon e vinagrete.'
data.loc[data['RESTAURANTE']=='Baixo Araguaia', 'ENDEREÇO'] = 'Rua Araguaia, 1709 | Freguesia de Jacarepaguá, Rio de Janeiro – RJ'
data.loc[data['RESTAURANTE']=='Baixo Araguaia', 'TELEFONE'] = '(21) 3392-3760'
data.loc[data['RESTAURANTE']=='Baixo Araguaia', 'HORARIOS'] = 'Segundas, das 18h às 24hTerças, das 18h às 24hQuartas, das 18h às 24hQuintas, das 18h às 24hSextas, das 18h às 24hSábados, das 12h às 24hDomingos, das 12h às 22h'

data.loc[data['RESTAURANTE']=='Bar da Gema', 'DESCRICAO'] = 'Deliciosa moela com bacon no molho da casa, coberta por uma polenta mole com farofa de bacon.'
data.loc[data['RESTAURANTE']=='Bar da Gema', 'ENDEREÇO'] = 'Rua Barão de Mesquita, 615 – Lojas C e D | Tijuca, Rio de Janeiro – RJ'
data.loc[data['RESTAURANTE']=='Bar da Gema', 'TELEFONE'] = '(21) 3549-0857'
data.loc[data['RESTAURANTE']=='Bar da Gema', 'HORARIOS'] = 'Terças, das 17h às 23hQuartas, das 17h às 23hQuintas, das 17h às 23hSextas, das 17h às 23hSábados, das 12h às 23hDomingos, das 12h às 17h'

In [8]:
data[data['PRATO']=='']

,RESTAURANTE,PRATO,DESCRICAO,ENDEREÇO,TELEFONE,HORARIOS
49,Carpaccio,,Siri CascudoBolinhos de carne de siri recheado...,"Rua Joana Kalil, 9 | Jardim Meriti, São João d...",(21) 2699-0468,"Horário: Segundas, das 12h às 16hTerças, das 1..."
62,Empório da Ovelha,,Cordeiro de Deus: Sabor dos DeusesMix de petis...,"Rua Álvares de Azevedo, 108 – Loja 102 | Icara...",(21) 3619-0449,"Horário: Terças, das 12h às 23hQuartas, das 12..."
76,Mr. Meat,,As Belas Coxas de IsabelDuas coxas de frango e...,"Rua Visconde de Abaeté, 139 | Vila Isabel, Rio...",(21) 96401-6410,"Horário: Terças, das 17h à 1hQuartas, das 17h ..."
82,Pasta da Nonna,,"Virando a LajeRisoto de arroz arbóreo, feito c...","Avenida Brasil, 367 – Duque de Caxias | Vila S...",(21) 3902-4780,"Horário: Segundas, das 18h às 24hTerças, das 1..."


In [9]:
data.loc[data['RESTAURANTE']=='Carpaccio', 'PRATO'] = 'Siri Cascudo'
data.loc[data['RESTAURANTE']=='Carpaccio', 'DESCRICAO'] = 'Bolinhos de carne de siri recheados com queijo provolone e empanados em farinha Panko.'

data.loc[data['RESTAURANTE']=='Empório da Ovelha', 'PRATO'] = 'Cordeiro de Deus: Sabor dos Deuses'
data.loc[data['RESTAURANTE']=='Empório da Ovelha', 'DESCRICAO'] = 'Mix de petiscos de cordeiro: 2 minikaftas + 2 bolinhas de linguiça + 2 croquetes, acompanhados de geleia de pimenta defumada e molho de hortelã.'

data.loc[data['RESTAURANTE']=='Mr. Meat', 'PRATO'] = 'As Belas Coxas de Isabel'
data.loc[data['RESTAURANTE']=='Mr. Meat', 'DESCRICAO'] = 'Duas coxas de frango envolvidas no bacon, defumadas e finalizadas no molho de barbecue e conhaque. Acompanhadas de batatas rústicas assadas com tempero de ervas e vinagrete de mix de pimentões e cebola roxa.'

data.loc[data['RESTAURANTE']=='Pasta da Nonna', 'PRATO'] = 'Virando a Laje'
data.loc[data['RESTAURANTE']=='Pasta da Nonna', 'DESCRICAO'] = 'Risoto de arroz arbóreo, feito com caldo de mocotó e recheado com carne seca desfiada, bacon, linguiça calabresa, costelinha de porco. Acompanha molho de rapadura apimentada e vinagrete de azeitona.'

In [18]:
data['NOTA'] = 1

data.loc[(data['DESCRICAO'].str.contains('lombo'))|(data['DESCRICAO'].str.contains('lombo')), 'NOTA'] = 2

data.loc[(data['DESCRICAO'].str.contains('carne seca'))|(data['DESCRICAO'].str.contains('Carne seca')), 'NOTA'] = 3

data.loc[(data['DESCRICAO'].str.contains('porco'))|(data['DESCRICAO'].str.contains('Porco')), 'NOTA'] = 4

data.loc[(data['DESCRICAO'].str.contains('camarão'))|(data['DESCRICAO'].str.contains('Camarão')), 'NOTA'] = 5

In [17]:
data.sort_values(by='NOTA', ascending=False).to_excel('Participantes comida de buteco.xlsx', index=False)

In [14]:
pd.set_option('display.max_rows', 500)
data.drop_duplicates()

,RESTAURANTE,PRATO,DESCRICAO,ENDEREÇO,TELEFONE,HORARIOS,NOTA
0,400 Steak Club,Pãozinho de Cupim,"Um delicioso pão de alho crocante, recheado co...","Rua Jambeiro, 400 – Loja | Vila Valqueire, Rio...",(21) 3579-1497,"Horário: Segundas, das 12h às 24hTerças, das 1...",1
1,Adega Pérola,Tilápia Mediterrânea,Tilápia empanada e molho mediterrâneo – acompa...,"Siqueira Campos, 138 – A | Copacabana, Rio de ...",(21) 2255-9425,"Horário: Segundas, das 11h à 1hTerças, das 11h...",1
2,Angurmê,Borogodó de Fubá à Baiana do Cais do Valongo,"Croquetes de fubá com miúdos bovinos, rabada e...","Rua Visconde de Abaeté, 123 | Vila Isabel, Rio...",(21) 99202-9229,"Horário: Quartas, das 12h às 24h Quintas, das ...",1
3,Areias de Itacoatiara,Mix Mineiro,2 Bolinhos de carne seca com requeijão e queij...,"Rua Barão de Mesquita, 615 – Lojas C e D | Tij...",(21) 3549-0857,"Terças, das 17h às 23hQuartas, das 17h às 23hQ...",3
4,Armazém São Thiago / Bar do Gomes,Hamburguinho de botequim,Hamburguinho de batata-doce e recheio de coste...,"Rua Áurea, 26 – Loja | Santa Teresa, Rio de Ja...",(21) 2232-0822,"Horário: Segundas, das 16h às 24hTerças, das 1...",4
5,Art Chopp,Nosso Porquinho deu Cuscuz!,"Copa lombo defumado em lenha de macieira, desf...","Rua Estrada Macembu, 63 | Taquara, Rio de Jane...",(21) 99566-8424,"Horário: Terças, das 17h à 0h30Quartas, das 17...",2
6,Baixo Araguaia,Entrevero Carioca,Espeto com deliciosa mistura de carnes e defum...,"Rua Araguaia, 1709 | Freguesia de Jacarepaguá,...",(21) 3392-3760,"Segundas, das 18h às 24hTerças, das 18h às 24h...",1
7,Baixo Gago,Amoela,"Moela à milanesa com crispy de couve, acompanh...","Rua Gago Coutinho, 51, 51 | Laranjeiras, Rio d...",(21) 2556-0638,"Horário: Segundas, das 11h às 22hTerças, das 1...",1
8,Balaio do Zé,Balainhos Confitados do Zé,Massa tenra crocante recheada com creme de que...,"Travessa Tamoios, 40 – Loja A | Flamengo, Rio ...",(21) 3796-5340,"Horário: Segundas, das 11h30 às 24hTerças, das...",1
9,Balbino’s – Delícia de Costela,Delícia de Costela,Croquetes de costela servidos com geleia de pi...,"Praça Do Bandolim, 296 | Curicica, Rio de Jane...",(21) 2441-0747,"Horário: Quartas, das 11h30 às 18hQuintas, das...",1


In [19]:
data

,RESTAURANTE,PRATO,DESCRICAO,ENDEREÇO,TELEFONE,HORARIOS,NOTA
0,400 Steak Club,Pãozinho de Cupim,"Um delicioso pão de alho crocante, recheado co...","Rua Jambeiro, 400 – Loja | Vila Valqueire, Rio...",(21) 3579-1497,"Horário: Segundas, das 12h às 24hTerças, das 1...",1
1,Adega Pérola,Tilápia Mediterrânea,Tilápia empanada e molho mediterrâneo – acompa...,"Siqueira Campos, 138 – A | Copacabana, Rio de ...",(21) 2255-9425,"Horário: Segundas, das 11h à 1hTerças, das 11h...",1
2,Angurmê,Borogodó de Fubá à Baiana do Cais do Valongo,"Croquetes de fubá com miúdos bovinos, rabada e...","Rua Visconde de Abaeté, 123 | Vila Isabel, Rio...",(21) 99202-9229,"Horário: Quartas, das 12h às 24h Quintas, das ...",1
3,Areias de Itacoatiara,Mix Mineiro,2 Bolinhos de carne seca com requeijão e queij...,"Rua Barão de Mesquita, 615 – Lojas C e D | Tij...",(21) 3549-0857,"Terças, das 17h às 23hQuartas, das 17h às 23hQ...",3
4,Armazém São Thiago / Bar do Gomes,Hamburguinho de botequim,Hamburguinho de batata-doce e recheio de coste...,"Rua Áurea, 26 – Loja | Santa Teresa, Rio de Ja...",(21) 2232-0822,"Horário: Segundas, das 16h às 24hTerças, das 1...",4
5,Art Chopp,Nosso Porquinho deu Cuscuz!,"Copa lombo defumado em lenha de macieira, desf...","Rua Estrada Macembu, 63 | Taquara, Rio de Jane...",(21) 99566-8424,"Horário: Terças, das 17h à 0h30Quartas, das 17...",2
6,Baixo Araguaia,Entrevero Carioca,Espeto com deliciosa mistura de carnes e defum...,"Rua Araguaia, 1709 | Freguesia de Jacarepaguá,...",(21) 3392-3760,"Segundas, das 18h às 24hTerças, das 18h às 24h...",1
7,Baixo Gago,Amoela,"Moela à milanesa com crispy de couve, acompanh...","Rua Gago Coutinho, 51, 51 | Laranjeiras, Rio d...",(21) 2556-0638,"Horário: Segundas, das 11h às 22hTerças, das 1...",1
8,Balaio do Zé,Balainhos Confitados do Zé,Massa tenra crocante recheada com creme de que...,"Travessa Tamoios, 40 – Loja A | Flamengo, Rio ...",(21) 3796-5340,"Horário: Segundas, das 11h30 às 24hTerças, das...",1
9,Balbino’s – Delícia de Costela,Delícia de Costela,Croquetes de costela servidos com geleia de pi...,"Praça Do Bandolim, 296 | Curicica, Rio de Jane...",(21) 2441-0747,"Horário: Quartas, das 11h30 às 18hQuintas, das...",1


In [25]:
data[(data['DESCRICAO'].str.contains('catupiry'))&(data['DESCRICAO'].str.contains('camarão'))]

,RESTAURANTE,PRATO,DESCRICAO,ENDEREÇO,TELEFONE,HORARIOS,NOTA
